In [2]:
import sys
sys.path.append('./codes/forgraph/')
from config import args
from sklearn.metrics import roc_auc_score
from models import GCN
from metrics import *
import numpy as np
from Explainer import Explainer
from scipy.sparse import coo_matrix,csr_matrix
import networkx as nx
skip = 5
topk = 5
import matplotlib
import matplotlib.pyplot as plt
import pickle as pkl


args.elr = 0.003
args.coff_t0=5.0
args.coff_t0=1.0
args.coff_size = 0.00
args.coff_ent = 0.000



2022-10-19 09:52:32.168040: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
with open('./dataset/BA-2motif.pkl','rb') as fin:
    adjs,features,labels = pkl.load(fin)

model = GCN(input_dim=features.shape[-1], output_dim=labels.shape[1])
model.load_weights(args.save_path+args.dataset)


embs = model.getNodeEmb((tf.convert_to_tensor(features,dtype=tf.float32),\
                               tf.convert_to_tensor(adjs,dtype=tf.float32)), training=False)
outputs = model.call((tf.convert_to_tensor(features,dtype=tf.float32),\
                            tf.convert_to_tensor(adjs,dtype=tf.float32)),training=False)
pred_label = tf.argmax(outputs, 1)

acc = accuracy(outputs, labels)
print('model acc',acc.numpy())

embs = model.getNodeEmb((tf.convert_to_tensor(features,dtype=tf.float32),\
                               tf.convert_to_tensor(adjs,dtype=tf.float32)), training=False)
outputs = model.call((tf.convert_to_tensor(features,dtype=tf.float32),\
                            tf.convert_to_tensor(adjs,dtype=tf.float32)),training=False)
pred_label = tf.argmax(outputs, 1)


model acc 1.0


2022-10-19 09:53:09.654991: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
if args.setting==1:
    allnodes = [i for i in range(0,100)]
elif args.setting==2:
    allnodes = [i for i in range(0,100)]
    allnodes.extend([i for i in range(500,600)])
elif args.setting==3:
    allnodes=[i for i in range(1000)]
explainer = Explainer(model=model,nodesize=adjs.shape[1])
optimizer = tf.keras.optimizers.Adam(learning_rate=args.elr)


In [5]:
def acc(adj,insert):
    mask = explainer.masked_adj.numpy()
    adj = coo_matrix(adj)
    for r,c in list(zip(adj.row,adj.col)):
        if r>=insert and r<insert+skip and c>=insert and c<insert+skip:
            reals.append(1)
        else:
            reals.append(0)
        preds.append(mask[r][c])

In [6]:
def explain_graph(gid):
    fea,emb,adj,label,graphid = features[gid], embs[gid], adjs[gid], labels[gid], gid
    explainer((fea,emb,adj,1.0,label))
    insert = 20
    acc(adj,insert)

In [7]:
def test():
    global preds
    global reals
    preds = []
    reals = []
    for gid in allnodes:
        explain_graph(gid)
    auc = roc_auc_score(reals,preds)
    return auc

In [8]:
def train():
    epochs = 10
    t0 = args.coff_t0
    t1 = args.coff_te

    for epoch in range(epochs):
        if epoch%1==0:
            auc = test()
            print('auc',auc)

            
        loss = 0
        tmp = float(t0 * np.power(t1 / t0, epoch /epochs))
        train_instances = [ins for ins in range(adjs.shape[0])]
        np.random.shuffle(train_instances)
        for gid in train_instances:
            with tf.GradientTape() as tape:
                pred = explainer((features[gid],embs[gid],adjs[gid],tmp, labels[gid]),training=True)
                loss += explainer.loss(pred, labels[gid])
                
#         train_variables = [para for para in explainer.trainable_variables
#                            if para.name.startswith('explainer')]
        train_variables = [para for para in explainer.trainable_variables]
        grads = tape.gradient(loss, train_variables)
        optimizer.apply_gradients(zip(grads, train_variables))
        print('epoch',epoch,'loss',loss.numpy())


In [9]:
train()

auc 0.011965620552699606
epoch 0 loss 1999.3518
auc 0.01479113117539426
epoch 1 loss 1520.0646
auc 0.016842902290087433
epoch 2 loss 1236.8398
auc 0.017821535280706347
epoch 3 loss 1090.1023
auc 0.01805955661322645
epoch 4 loss 1052.2131
auc 0.018722113611280535
epoch 5 loss 1064.1089
auc 0.01914778379222212
epoch 6 loss 1090.7587
auc 0.01987239878307339
epoch 7 loss 1155.9995
auc 0.02340970437251315
epoch 8 loss 1221.5469
auc 0.0443627290813511
epoch 9 loss 1314.2928


In [10]:
auc = test()
print(auc)

0.067249081496326


In [11]:
gid = 501
topk=6
fea,emb,adj,label,graphid = features[gid], embs[gid], adjs[gid], labels[gid], gid
explainer((fea,emb,adj,1.0,label))
plot(adj,label,graphid)


NameError: name 'plot' is not defined

In [ ]:
gid = 1
topk=5
fea,emb,adj,label,graphid = features[gid], embs[gid], adjs[gid], labels[gid], gid
explainer((fea,emb,adj,1.0,label))
plot(adj,label,graphid)